In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [6]:
test_seq = torch.randn(20,5)

In [7]:
test_seq

tensor([[ 1.1178,  0.6974,  0.3317, -1.5489,  1.7258],
        [-1.6983,  0.0571,  1.0761, -0.9370, -0.0728],
        [ 0.0167,  0.7060, -0.8473,  1.7103, -0.2992],
        [-1.3477, -1.1205,  0.8219, -1.0260, -0.1550],
        [-1.1963, -0.2946, -0.3406, -1.5448, -0.8546],
        [-0.0034, -1.6447,  0.3536,  0.6514, -0.5334],
        [ 0.9225, -1.5803, -0.4260,  0.0374, -1.2610],
        [ 1.1278,  0.2030, -1.1835, -0.6342,  0.4107],
        [-0.2629, -1.1219,  0.8876,  0.9207, -0.6915],
        [-1.9146, -0.5393,  0.4731,  0.8181, -0.6927],
        [ 1.3584, -1.0825, -0.7219, -0.2689, -1.9217],
        [-0.1313,  0.5677,  1.3095, -0.4247, -2.5669],
        [ 0.9901, -1.0375, -1.0113, -0.0737,  1.1509],
        [-1.3734,  0.6485, -0.2309,  1.5850,  0.5285],
        [-0.8151,  0.1861, -1.1545,  2.1413, -1.1514],
        [ 0.6874,  0.5364,  0.9465, -0.8875, -0.0218],
        [ 1.0845,  1.5818,  1.2209,  0.1913, -0.1972],
        [ 1.1928,  0.9025,  0.1889, -0.4885,  0.1036],
        [ 

In [13]:
conv1 = nn.Conv1d(5, 4, 3)

In [16]:
list(conv1.parameters())

[Parameter containing:
 tensor([[[-0.0464, -0.0211, -0.1917],
          [-0.0735, -0.1753, -0.1654],
          [ 0.1133,  0.1441,  0.0713],
          [ 0.2496,  0.0801, -0.1862],
          [-0.2355, -0.2356, -0.2059]],
 
         [[ 0.1150, -0.1785, -0.1576],
          [-0.1370,  0.1081, -0.1956],
          [ 0.0371, -0.0190, -0.1188],
          [-0.2198, -0.1322,  0.1014],
          [ 0.1671,  0.1161,  0.2557]],
 
         [[ 0.1957, -0.0978, -0.0660],
          [ 0.1532, -0.2031, -0.1964],
          [-0.2421,  0.0702,  0.2486],
          [-0.0315, -0.2399,  0.1663],
          [-0.0529, -0.1911, -0.1765]],
 
         [[-0.0689,  0.0169, -0.1938],
          [ 0.1249,  0.0672, -0.1173],
          [-0.0577, -0.1353,  0.1382],
          [-0.0589,  0.1761,  0.0110],
          [-0.0192,  0.0180, -0.1651]]]), Parameter containing:
 tensor([ 0.0069,  0.1292,  0.1128, -0.2523])]

In [21]:
test_seq.shape # L, C_in

torch.Size([20, 5])

In the simplest case, the output value of the layer with input size
$(N, C_{in}, L)$ and output $(N, C_{out}, L_{out})$ can be
precisely described as:


In [23]:
input_seq = torch.unsqueeze(test_seq.transpose(0,1),0)
input_seq.shape

torch.Size([1, 5, 20])

In [28]:
out_seq = conv1(input_seq)
out_seq.shape

torch.Size([1, 4, 18])

In [30]:
max_pool = torch.nn.MaxPool1d(3)

In [32]:
pooled_seq = max_pool(out_seq)
pooled_seq.shape

torch.Size([1, 4, 6])

In [34]:
max_pool2 = torch.nn.MaxPool1d(3, stride=1)

In [35]:
pooled_seq2 = max_pool2(out_seq)
pooled_seq2.shape

torch.Size([1, 4, 16])

In [36]:
embedding = nn.Embedding(10,300)

In [40]:
out_embed1 = embedding(torch.tensor([1,2,3]))
out_embed2 = embedding(torch.tensor([[1,2,3],[4,5,6]]))
out_embed1.shape, out_embed2.shape

(torch.Size([3, 300]), torch.Size([2, 3, 300]))

In [43]:
torch.cat([torch.randn(2,3), torch.randn(2,3)],0).shape

torch.Size([4, 3])

In [127]:
class CNN1DText(nn.Module):
    def __init__(self, embed_num = 259922, embed_dim = 300, class_num = 2, kernel_num = 100, kernel_sizes = (3,4,5),
                dropout = 0.5):
        super(CNN1DText, self).__init__()
        
        self.embed_num = embed_num
        self.embed_dim = embed_dim
        self.class_num = class_num
        self.kernel_num = kernel_num
        self.kernel_sizes = kernel_sizes
        
        min_size = max(kernel_sizes)
        self.pad_sizes = [((size - min_size)//2 + (size -min_size)%2 , (size -min_size)//2)  for size in kernel_sizes]
        
        self.embed = nn.Embedding(embed_num, embed_dim)
        self.convs = nn.ModuleList([nn.Conv1d(embed_dim, kernel_num, kernel_size) for kernel_size in kernel_sizes])
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(kernel_num * len(kernel_sizes), class_num) # fake fc
    def batch_forward(self, x):
        # x (L)
        x = self.embed(x) # (L, embed_dim)
        x = torch.unsqueeze(x.transpose(0,1),0) # (1, embed_dim, L)
        x = [conv(x) for conv in self.convs] # [(1, kernel_num, L'), (1, kernel_num, L''), ...]
        x = [F.pad(torch.squeeze(x[i], 0), self.pad_sizes[i]) for i in range(len(x))] # [(kernel_num, L'),...]
        x = torch.cat(x,0) # (len(kernel_sizes)*kernel_num, L')
        x = x.transpose(0,1) # L', kernel_num, len(kernel_sizes) 
        x = self.fc(x) # (L', class_num)
        return x # logit. softmax(x) = probability output 
    def forward(self, x):
        # add and remove the extra N=1 dummy dimention to leverage other procedure which can only handle such form, 
        # such as those traning algorithm.
        # (1, L)
        x = self.batch_forward(torch.squeeze(x,0))
        return torch.unsqueeze(torch.max(x,0)[0],0) #(1, class_num)

In [128]:
model = CNN1DText(embed_num = 259922, embed_dim = 300, class_num = 2, kernel_num = 100, kernel_sizes = (3,4,5),
                dropout = 0.5)

In [129]:
fake_input = torch.tensor([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

In [130]:
torch.unsqueeze(fake_input,0).shape

torch.Size([1, 11])

In [131]:
model(torch.unsqueeze(fake_input,0))

tensor([[ 0.2715,  0.1266]])

In [132]:
F.softmax(model(fake_input), 1)

tensor([[ 0.5362,  0.4638]])

In [133]:
model.batch_forward(fake_input)

tensor([[ 0.2364,  0.0367],
        [ 0.2715,  0.1266],
        [ 0.0579, -0.4812],
        [ 0.1000, -0.5245],
        [-0.1228,  0.1191],
        [ 0.2532,  0.0634],
        [ 0.0963,  0.1197]])

In [134]:
F.softmax(model.batch_forward(fake_input), 1)

tensor([[ 0.5498,  0.4502],
        [ 0.5362,  0.4638],
        [ 0.6316,  0.3684],
        [ 0.6512,  0.3488],
        [ 0.4398,  0.5602],
        [ 0.5473,  0.4527],
        [ 0.4941,  0.5059]])